In [1]:
import tensorflow as tf

try:
    interpreter = tf.lite.Interpreter(model_path="ted_tflite_models/ted_model.tflite")
    interpreter.allocate_tensors()
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")

2024-08-24 22:23:46.011903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-24 22:23:46.029495: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-24 22:23:46.029527: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-24 22:23:46.040101: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-24 22:23:47.179200: W tensorflow/compiler/tf

Model loaded successfully.


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [2]:
# Получаем информацию о всех тензорах
tensor_details = interpreter.get_tensor_details()

# Поиск и вывод информации о tensor#58
tensor_index = 53  # Это индекс тензора, указанный в предупреждении

for tensor in tensor_details:
    if tensor['index'] == tensor_index:
        print(f"Tensor found: {tensor}")
        break
else:
    print(f"Tensor with index {tensor_index} not found.")

Tensor found: {'name': 'ted/up_conv_block_2/sequential_2/conv2d_transpose_2/BiasAdd;ted/up_conv_block_2/sequential_2/conv2d_transpose_2/conv2d_transpose;ted/up_conv_block_2/sequential_2/conv2d_transpose_2/BiasAdd/ReadVariableOp', 'index': 53, 'shape': array([  1, 176, 176,  16], dtype=int32), 'shape_signature': array([  1, 176, 176,  16], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


In [3]:
import pathlib

model = tf.keras.layers.TFSMLayer('best_model', call_endpoint='serving_default')

fixed_input = tf.keras.Input(shape=(352, 352, 3), batch_size=1)

fixed_model = tf.keras.Model(inputs=fixed_input, outputs=model(fixed_input))

converter = tf.lite.TFLiteConverter.from_keras_model(fixed_model)
tflite_model = converter.convert()

tflite_models_dir = pathlib.Path("ted_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

tflite_model_file = tflite_models_dir/"ted_model.tflite"
tflite_model_file.write_bytes(tflite_model)


2024-08-24 22:24:02.712953: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-24 22:24:03.042975: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-24 22:24:03.043429: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

INFO:tensorflow:Assets written to: /tmp/tmpstciv092/assets


INFO:tensorflow:Assets written to: /tmp/tmpstciv092/assets
W0000 00:00:1724527446.749908   12233 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1724527446.749965   12233 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-08-24 22:24:06.751286: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpstciv092
2024-08-24 22:24:06.771984: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-08-24 22:24:06.772021: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpstciv092
2024-08-24 22:24:06.833077: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-08-24 22:24:06.839673: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-08-24 22:24:07.091293: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: /tmp/tmpstciv092
2024-0

246164

In [4]:
import onnx

onnx.utils.extract_model("teed_model.onnx", "teed_model_subgraph2_extracted.onnx", ["input"] , ["output2"], True)

In [5]:
import onnxruntime as rt

sess_options = rt.SessionOptions()

# Set graph optimization level
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_EXTENDED

# To enable model serialization after graph optimization set this
sess_options.optimized_model_filepath = "optimized_subgraph2.onnx"

session = rt.InferenceSession("teed_model_subgraph2_extracted.onnx", sess_options)

In [6]:
import onnx
import numpy as np
from onnx import numpy_helper
from sklearn.cluster import KMeans
from onnx import helper

# Загрузка модели
model_path = "optimized_subgraph2.onnx"
model = onnx.load(model_path)



# Извлечение весов из модели
weights = []
weight_names = []
for initializer in model.graph.initializer:
    tensor = numpy_helper.to_array(initializer)
    if tensor.ndim > 1:  # Кластеризация применима к многомерным весам
        weights.append(tensor)
        weight_names.append(initializer.name)

# Выполнение кластеризации весов
max_clusters = 16  # Максимальное количество кластеров
clustered_weights = []

for w in weights:
    original_shape = w.shape
    w_flattened = w.flatten().reshape(-1, 1)
    
    # Определяем количество кластеров как минимум между max_clusters и уникальными значениями в весах
    n_clusters = min(max_clusters, len(np.unique(w_flattened)))

    # Выполнение KMeans кластеризации
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(w_flattened)
    
    # Замена весов на ближайшие центроиды
    new_weights = kmeans.cluster_centers_[kmeans.labels_]
    clustered_weights.append(new_weights.reshape(original_shape))

# Замена весов в модели
new_initializers = []
for i, initializer in enumerate(model.graph.initializer):
    if initializer.name in weight_names:
        new_tensor = clustered_weights.pop(0)
        new_initializer = numpy_helper.from_array(new_tensor, initializer.name)
        new_initializers.append(new_initializer)
    else:
        new_initializers.append(initializer)

# Очистка текущего списка инициализаторов и добавление новых
del model.graph.initializer[:]
model.graph.initializer.extend(new_initializers)

# Сохранение обновленной модели
onnx.save(model, "model_clustered2.onnx")

# Тестирование модели с onnxruntime (опционально)
import onnxruntime as ort

session = ort.InferenceSession("model_clustered2.onnx")
# Загрузите входные данные и выполните инференс
# input_name = session.get_inputs()[0].name
# output_name = session.get_outputs()[0].name
# result = session.run([output_name], {input_name: input_data})

In [11]:
from PIL import Image
import torch
from torchvision import transforms
import onnxruntime as ort

# Загрузка изображения
image_path = "PIXECT-20240826122749.jpg"
image = Image.open(image_path)

# Определение преобразований
transform = transforms.Compose([
    transforms.Resize((352, 352)),  # Изменение размера изображения до 352x352
    transforms.ToTensor(),  # Преобразование в тензор
    transforms.Normalize(mean=[104.007, 116.669, 122.679], std=[0.229, 0.224, 0.225])  # Нормализация
])

# Применение преобразований к изображению
image_tensor = transform(image)

# Добавление измерения batch_size (например, batch_size=1)
image_tensor = image_tensor.unsqueeze(0)

# Преобразование тензора в numpy для подачи в ONNX Runtime
image_numpy = image_tensor.numpy()

# Загрузка модели ONNX с профилированием
model_path = "model_clustered.onnx"
sess_options = ort.SessionOptions()
sess_options.enable_profiling = True  # Включение профилирования
session = ort.InferenceSession(model_path, sess_options)

# Получение имени входного и выходного тензоров
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

# Выполнение инференса с профилированием
output = session.run([output_name], {input_name: image_numpy})

# Завершение профилирования и получение пути к профилю
profile_file = session.end_profiling()
print(f"Profile saved to {profile_file}")

# Дополнительно: можно проанализировать профиль, загрузив его в инструменты визуализации, такие как Chrome Tracing.


Profile saved to onnxruntime_profile__2024-08-27_17-01-05.json


In [10]:
!./linux_x86-64_benchmark_model --graph=ted_model_new.tflite --num_threads=4 --enable_op_profiling=true > profiling_correct.log

INFO: STARTING!
INFO: Log parameter values verbosely: [0]
INFO: Num threads: [4]
INFO: Graph: [ted_model_new.tflite]
INFO: Signature to run: []
INFO: Enable op profiling: [1]
INFO: #threads used for CPU inference: [4]
INFO: Loaded model ted_model_new.tflite
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: The input model file size (MB): 0.146744
INFO: Initialized session in 73.469ms.
INFO: Running benchmark for at least 1 iterations and at least 0.5 seconds but terminate if exceeding 150 seconds.
INFO: count=9 first=116884 curr=49696 min=49424 max=116884 avg=58081.7 std=20810

INFO: Running benchmark for at least 50 iterations and at least 1 seconds but terminate if exceeding 150 seconds.
INFO: count=50 first=49772 curr=49450 min=48059 max=63538 avg=51197.9 std=2561

INFO: Inference timings in us: Init: 73469, First inference: 116884, Warmup (avg): 58081.7, Inference (avg): 51197.9
INFO: Note: as the benchmark tool itself affects memory footprint, the following is only APP

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

# Создание DataFrame с вашими данными
data = {
    "Node Type": [
        "Convolution (NHWC, F32) IGEMM",
        "TRANSPOSE_CONV",
        "Add (ND, F32)",
        "TRANSPOSE_CONV",
        "TRANSPOSE_CONV",
        "TRANSPOSE_CONV",
        "Convolution (NHWC, F32) DWConv",
        "SUM",
        "Clamp (NC, F32)",
        "Copy (NC, X32)"
    ],
    "First (ms)": [7.311, 4.477, 4.115, 4.248, 4.306, 4.407, 3.575, 3.297, 2.939, 2.456],
    "Avg (ms)": [7.429, 4.318, 4.146, 3.732, 3.709, 3.674, 3.611, 3.349, 3.002, 2.624],
    "% Time": ["14.478%", "8.415%", "8.081%", "7.274%", "7.228%", "7.161%", "7.038%", "6.527%", "5.851%", "5.115%"],
    "CDF%": ["14.478%", "22.893%", "30.974%", "38.248%", "45.477%", "52.637%", "59.675%", "66.202%", "72.053%", "77.168%"],
    "Mem (KB)": [0.000] * 10,
    "Times Called": [1] * 10,
    "Name": [
        "Delegate/Convolution (NHWC, F32) IGEMM:2",
        "[ted_5_1/up_conv_block_17_1/sequential_17_1/conv2d_transpose_22_1/add;ted_5_1/up_conv_block_17_1/sequential_17_1/conv2d_transpose_22_1/conv_transpose;]:32",
        "Delegate/Add (ND, F32):5",
        "[StatefulPartitionedCall_1:2]:37",
        "[StatefulPartitionedCall_1:1]:27",
        "[StatefulPartitionedCall_1:0]:22",
        "Delegate/Convolution (NHWC, F32) DWConv:4",
        "[ted_5_1/double_fusion_5_1/Sum]:44",
        "Delegate/Clamp (NC, F32):3",
        "Delegate/Copy (NC, X32):0"
    ]
}

df = pd.DataFrame(data)

# Создание пользовательской цветовой карты от белого к красному
cmap = LinearSegmentedColormap.from_list('white_red', ['white', 'red'])

# Применение условного форматирования к DataFrame
def color_scale(val):
    return f'background-color: {cmap(val / df["Avg (ms)"].max())}'

styled_df = df.style.applymap(color_scale, subset=['Avg (ms)'])
styled_df


/tmp/ipykernel_15704/1498892007.py:49: FutureWarning:

Styler.applymap has been deprecated. Use Styler.map instead.



,Node Type,First (ms),Avg (ms),% Time,CDF%,Mem (KB),Times Called,Name
0,"Convolution (NHWC, F32) IGEMM",7.311000,7.429000,14.478%,14.478%,0.000000,1,"Delegate/Convolution (NHWC, F32) IGEMM:2"
1,TRANSPOSE_CONV,4.477000,4.318000,8.415%,22.893%,0.000000,1,[ted_5_1/up_conv_block_17_1/sequential_17_1/conv2d_transpose_22_1/add;ted_5_1/up_conv_block_17_1/sequential_17_1/conv2d_transpose_22_1/conv_transpose;]:32
2,"Add (ND, F32)",4.115000,4.146000,8.081%,30.974%,0.000000,1,"Delegate/Add (ND, F32):5"
3,TRANSPOSE_CONV,4.248000,3.732000,7.274%,38.248%,0.000000,1,[StatefulPartitionedCall_1:2]:37
4,TRANSPOSE_CONV,4.306000,3.709000,7.228%,45.477%,0.000000,1,[StatefulPartitionedCall_1:1]:27
5,TRANSPOSE_CONV,4.407000,3.674000,7.161%,52.637%,0.000000,1,[StatefulPartitionedCall_1:0]:22
6,"Convolution (NHWC, F32) DWConv",3.575000,3.611000,7.038%,59.675%,0.000000,1,"Delegate/Convolution (NHWC, F32) DWConv:4"
7,SUM,3.297000,3.349000,6.527%,66.202%,0.000000,1,[ted_5_1/double_fusion_5_1/Sum]:44
8,"Clamp (NC, F32)",2.939000,3.002000,5.851%,72.053%,0.000000,1,"Delegate/Clamp (NC, F32):3"
9,"Copy (NC, X32)",2.456000,2.624000,5.115%,77.168%,0.000000,1,"Delegate/Copy (NC, X32):0"
